<a href="https://colab.research.google.com/github/ElviraKonovalov/soen471-bigData/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [239]:
import pandas as pd

In [240]:
file = '../data/Motor Vehicle Collisions with KSI Data.csv'
df=  pd.read_csv(file, sep=',');

/tmp/ipykernel_59922/1997809495.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=  pd.read_csv(file, sep=',');


## DROPPING COLUMNS
We start by dropping the columns that we deem not pertinent for our model.

In [241]:
columns_to_drop = ['_id', 'ACCNUM', 'TIME', 'OFFSET', 'WARDNUM', 'DIVISION', 'ACCLOC', 'FATAL_NO', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'POLICE_DIVISION', 'ObjectId', 'geometry','DISTRICT','DRIVACT','DRIVCOND','IMPACTYPE','INJURY','NEIGHBOURHOOD','STREET1', 'STREET2','VEHTYPE','YEAR']
df = df.drop(columns=columns_to_drop)

## FILTERING
Second, since we're only interested in the classification for the driver, we filter all the rows that did not have a driver as involved party. Then since our data only contains drivers, the INVTYPE feature is redudant and we drop it.

In [242]:
df.INVTYPE.fillna('Unknown', inplace=True)

for row in df.INVTYPE.unique():
  if 'Driver' not in row or 'Not Hit' in row:
    df = df[(df.INVTYPE != row)]

df.drop(columns=['INVTYPE'], inplace=True)

## REPLACING BINARY VALUES

The following features have been noted to have a binary value; either "Yes" or "null", therefore, for each we map the values to 1 and 0 respectively.

In [243]:
df.AG_DRIV.fillna(0, inplace=True)
df.AG_DRIV.replace('Yes', 1, inplace=True)

df.ALCOHOL.fillna(0, inplace=True)
df.ALCOHOL.replace('Yes', 1, inplace=True)

df.AUTOMOBILE.fillna(0, inplace=True)
df.AUTOMOBILE.replace('Yes',1, inplace=True)

df.CYCLIST.fillna(0, inplace=True)
df.CYCLIST.replace('Yes',1, inplace=True)

df.DISABILITY.fillna(0, inplace=True)
df.DISABILITY.replace('Yes', 1, inplace=True)

df.EMERG_VEH.fillna(0, inplace=True)
df.EMERG_VEH.replace('Yes',1, inplace=True)

df.PEDESTRIAN.fillna(0, inplace=True)
df.PEDESTRIAN.replace('Yes',1, inplace=True)

df.PASSENGER.fillna(0, inplace=True)
df.PASSENGER.replace('Yes',1, inplace=True)

df.MOTORCYCLE.fillna(0, inplace=True)
df.MOTORCYCLE.replace('Yes',1, inplace=True)

df.REDLIGHT.fillna(0, inplace=True)
df.REDLIGHT.replace('Yes', 1, inplace=True)

df.SPEEDING.fillna(0, inplace=True)
df.SPEEDING.replace('Yes', 1, inplace=True)

df.TRSN_CITY_VEH.fillna(0, inplace=True)
df.TRSN_CITY_VEH.replace('Yes', 1, inplace=True)

df.TRUCK.fillna(0, inplace=True)
df.TRUCK.replace('Yes', 1, inplace=True)

Similarly, ACCLASS, representing the class of the accident is reduced to only 2 values, we represent fatal ones by 1 and non-fatal ones by 0.

In [244]:
df.ACCLASS.replace(['Fatal'], 1, inplace=True)
df.ACCLASS.replace(['Non-Fatal Injury', 'Property Damage Only'], 0, inplace=True)

## RENAMING AND GROUPING
We now group certain information together be renaming it to a common value.

We have decided that we didn't have to be too precise with the date, and that the month would suffice. Our assumption is that the occurence of accidents can be related to the holidays and the different weather conditions through the year.

In [245]:
#first we will have to convert the data type to DateTime
df['DATE'] = pd.to_datetime(df['DATE'])

#now we will transform the data in the DATE column to simply present the month in integer form
df['DATE'] = df['DATE'].dt.month

Next, Every street owned by the City of Toronto has been given one of five classifications (with the exception of public laneways): Local road, Collector road, Minor arterial road, Major arterical road and Expressway
https://www.toronto.ca/services-payments/streets-parking-transportation/traffic-management/road-classification-system/about-the-road-classification-system/

Therefore, to simplify our dataset, we renamed our values such that they fall into one the of the 5 classes. Additionally, to fill in null values we have selected the most frequent road class and we have droppend roads that were classed as "Pending" as they might fall into an outlier category of roads which don't fall exactly into one of the 5 classes.

In [246]:
df['ROAD_CLASS'] = df['ROAD_CLASS'].fillna(df['ROAD_CLASS'].value_counts().index[0])
# drop rows where ROAD_CLASS is pending
df.drop(df.index[df['ROAD_CLASS'] == 'Pending'], inplace=True)
# replacing Expressway Ramp (4 rows) with Expressway
df['ROAD_CLASS'] = df['ROAD_CLASS'].replace(['Expressway Ramp'],'Expressway')
# Major Arterial Ramp (1 row) with Major Arterial
df['ROAD_CLASS'] = df['ROAD_CLASS'].replace(['Major Arterial Ramp'],'Major Arterial')

We also group traffic control into 3 categories, namely: "Active Control", "Passive Control" and "No Control".
Active Control refers to traffic being controlled by a person such as a police officer, school guard or traffic controller.
Passive Control refers to traffic being controlled by a road sign such as a Stop sign or traffic signals.
No Control refers to traffic deprived of any control mechanism.

Our assumption for null values is that there must be a high probability of some form of passive control such as a road sign.

In [247]:
df['TRAFFCTL'] = df['TRAFFCTL'].replace(['Traffic Signal', 'Stop Sign', 'Pedestrian Crossover', 'Yield Sign', 'Streetcar (Stop for)', 'Traffic Gate'], 'Passive Control')
df['TRAFFCTL'] = df['TRAFFCTL'].replace(['Police Control', 'School Guard', 'Traffic Controller'], 'Active Control')
df['TRAFFCTL'] = df['TRAFFCTL'].fillna('Passive Control')

## RANDOM CLEANING
We noticed some of the values for INVAGE weren't uniform, therefore we renamed it with the right capitalization.

In [248]:
#All of the unknowns in this dataset are upper case except the ones in this column.
df.INVAGE = df.INVAGE.replace(['unknown'], 'Unknown')

## DROPPING ROWS
Since we deem the features left important to our models, we didn't want to leave any ambiguity in the data and drop those rows.
The dropped rows amounted to 475 which represent 5.547% our cleaned data which is a relatively low value.

In [249]:
df.drop(df[df.LOCCOORD.isnull()].index, inplace=True)

df.drop(df[df.INVAGE == 'Unknown'].index, inplace=True)

df.INITDIR.fillna("Unknown", inplace = True)
df.drop(df[df.INITDIR == 'Unknown'].index, inplace=True)

df.drop(df[df.MANOEUVER == 'Unknown'].index, inplace=True)
df.drop(df[df.MANOEUVER.isnull()].index, inplace=True)

df.drop(df[df.LIGHT == 'Other'].index, inplace=True)
df.drop(df[df.LIGHT.isnull()].index, inplace=True)

df.drop(df[df.RDSFCOND.isnull()].index, inplace=True)

df.drop(df[df.VISIBILITY.isnull()].index, inplace=True)

## SAVING THE DATA

In [250]:
df
# save data to csv file
df.to_csv("../data/clean_data.csv")

## HOT-ONE ENCODING

In [251]:
df['ACCLASS'] = df['ACCLASS'].astype('category').cat.codes
df['INITDIR'] = df['INITDIR'].astype('category').cat.codes
df['LIGHT'] = df['LIGHT'].astype('category').cat.codes
df['VISIBILITY'] = df['VISIBILITY'].astype('category').cat.codes
df['RDSFCOND'] = df['RDSFCOND'].astype('category').cat.codes
df['ROAD_CLASS'] = df['ROAD_CLASS'].astype('category').cat.codes # label encoding
df['TRAFFCTL'] = df['TRAFFCTL'].astype('category').cat.codes
df['INVAGE'] = df['INVAGE'].astype('category').cat.codes
df['LOCCOORD'] = df['LOCCOORD'].astype('category').cat.codes
df['MANOEUVER'] = df['MANOEUVER'].astype('category').cat.codes

In [252]:
df
# save data to csv file with hot-one encoding
df.to_csv("../data/clean_data_hot-one.csv")